# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('city_data.csv')
weather_df.head()


,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Norman Wells,65.2820,-126.8329,265.15,41,75,1.54
1,1,New Norfolk,-42.7826,147.0587,289.04,49,4,1.79
2,2,Rikitea,-23.1203,-134.9692,298.76,72,100,3.26
3,3,Nouakchott,18.0858,-15.9785,294.15,83,52,5.14
4,4,Upernavik,72.7868,-56.1549,274.09,77,100,2.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
humidity = weather_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = weather_df[["Latitude", "Longitude"]]

In [31]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
#I realized after the fact that my data pulled in kelvin for some reason but I fixed the original code. 

city_df = weather_df.loc[(weather_df["Temperature"] > 300) & (weather_df["Temperature"] < 320) & (weather_df["Cloudiness"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)

del city_df['index']

city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,157,Gouré,13.9835,10.2704,300.70,11,0,3.27
1,240,Kibala,9.1104,18.3463,303.15,51,0,2.21
2,356,Karratha,-20.7377,116.8463,301.02,21,0,6.34
3,422,Garhi Khairo,28.0606,67.9803,303.64,19,0,4.08
4,450,Dalmau,26.0667,81.0333,301.30,6,0,2.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotellist = []

for i in range(len(city_df)):
    lat = city_df.loc[i]['Latitude']
    lng = city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
city_df["Hotel Name"] = hotellist
city_df = city_df.dropna(how='any')

hotel_df = city_df

hotel_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,157,Gouré,13.9835,10.2704,300.70,11,0,3.27,Gouré
1,240,Kibala,9.1104,18.3463,303.15,51,0,2.21,Sarh
2,356,Karratha,-20.7377,116.8463,301.02,21,0,6.34,Karratha
3,422,Garhi Khairo,28.0606,67.9803,303.64,19,0,4.08,Garhi Khairo
4,450,Dalmau,26.0667,81.0333,301.30,6,0,2.52,Dalmau


In [37]:
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["latitude"]
    lng = row["longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        #hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

KeyError: 'latitude'

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [18]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

NameError: name 'locations' is not defined